In [ ]:
"""
Factor Timing Strategy Grid Search

This program performs a grid search across two dimensions:
1. Number of factors: 1, 3, 5, 7, 10, 15
2. Lookback periods: 24m, 36m, 60m, 72m, 90m

It displays three grids to the terminal:
1. Annualized returns for each combination
2. Sharpe ratios for each combination
3. Monthly turnover for each combination

IMPORTANT: This program only uses factors that have Max=1 in Step Factor Categories.xlsx.
Factors with Max=0 are excluded from consideration.

This helps identify the optimal combination of factor count and lookback period.
"""

import pandas as pd
import numpy as np
import time
from tabulate import tabulate

def factor_timing_strategy(excel_path: str, n_top_factors: int = 3, lookback: int = 36, allowed_factors: list = None) -> tuple[pd.Series, pd.DataFrame, pd.DataFrame]:
    """
    Implements a factor timing strategy that selects top performing factors based on recent performance metrics.
    Weights are assigned proportionally to Sharpe ratios rather than equal weighting.
    
    Parameters
    ----------
    excel_path : str
        Path to Excel file containing factor returns data
    n_top_factors : int, default=3 
        Number of top factors to select each month
    lookback : int, default=36
        Number of months to look back for calculating factor scores
    allowed_factors : list, default=None
        List of factor names to consider. If None, all factors are used.
    """
    
    # Read data
    df = pd.read_excel(excel_path)
    df.index = pd.to_datetime(df.iloc[:,0])
    returns_df = df.iloc[:,1:].astype(float) / 100
    
    # Filter for allowed factors if specified
    if allowed_factors is not None:
        # Keep only factors in the allowed_factors list
        available_factors = set(returns_df.columns)
        filtered_factors = [f for f in allowed_factors if f in available_factors]
        
        if len(filtered_factors) == 0:
            raise ValueError(f"None of the allowed factors found in {excel_path}. Check factor names match exactly.")
            
        returns_df = returns_df[filtered_factors]
        print(f"Filtered returns data to {len(filtered_factors)} allowed factors.")
    
    # Initialize factor scores dataframe
    factor_scores = pd.DataFrame(0.0, index=returns_df.index, columns=returns_df.columns)
    
    # Calculate scores for each date
    for date in returns_df.index[lookback:]:
        hist_data = returns_df.loc[:date]
        
        for factor in returns_df.columns:
            # Calculate components of factor score
            momentum = hist_data[factor].tail(lookback).mean()
            hit_rate = (hist_data[factor].tail(lookback) > 0).mean()
            vol = hist_data[factor].tail(lookback).std()
            sharpe = momentum / vol if vol != 0 else 0
            
            # Combined score weights momentum, hit rate and risk-adjusted return
            factor_scores.loc[date, factor] = momentum * hit_rate * (1 + sharpe)
    
    # Generate positions
    positions = pd.DataFrame(0.0, index=returns_df.index, columns=returns_df.columns)
    
    for date in factor_scores.index[lookback:]:
        # Select factors with positive scores
        positive_factors = factor_scores.loc[date][factor_scores.loc[date] > 0]
        
        if len(positive_factors) > 0:
            # Take top N factors
            top_factors = positive_factors.nlargest(min(n_top_factors, len(positive_factors)))
            
            # Calculate Sharpe ratios for top factors
            sharpe_ratios = {}
            for factor in top_factors.index:
                hist_data = returns_df[factor].loc[:date].tail(lookback)
                momentum = hist_data.mean()
                vol = hist_data.std()
                sharpe = momentum / vol if vol != 0 else 0
                sharpe_ratios[factor] = max(sharpe, 0)  # Only use positive Sharpe ratios
            
            # Calculate weights proportional to Sharpe ratios
            total_sharpe = sum(sharpe_ratios.values())
            if total_sharpe > 0:
                for factor, sharpe in sharpe_ratios.items():
                    positions.loc[date, factor] = sharpe / total_sharpe
    
    # Calculate strategy returns using previous month's weights
    strategy_returns = pd.Series(index=returns_df.index[lookback+1:], dtype=float)
    dates = returns_df.index
    
    for i, date in enumerate(dates[lookback+1:], lookback+1):
        prev_date = dates[dates < date][-1]  # Get the previous month's date
        # Calculate returns using previous month's weights
        strategy_returns[date] = np.sum(
            positions.loc[prev_date] * returns_df.loc[date]
        )
    
    return strategy_returns, positions, factor_scores

def calculate_performance_metrics(returns: pd.Series, positions: pd.DataFrame) -> tuple[float, float, float]:
    """
    Calculate annualized return, Sharpe ratio, and turnover for a return series.
    
    Parameters
    ----------
    returns : pd.Series
        Monthly returns of the strategy
    positions : pd.DataFrame
        Factor weights/positions over time
        
    Returns
    -------
    tuple
        (annualized_return, sharpe_ratio, turnover)
    """
    monthly_mean = returns.mean()
    monthly_vol = returns.std()
    
    ann_return = (1 + monthly_mean)**12 - 1
    ann_vol = monthly_vol * np.sqrt(12)
    sharpe = ann_return / ann_vol if ann_vol != 0 else 0
    
    # Calculate turnover (average monthly absolute change in positions)
    turnover = positions.diff().abs().sum(axis=1).mean()
    
    return ann_return, sharpe, turnover

def run_grid_search(excel_path: str, n_factors_list: list[int], lookbacks_list: list[int], allowed_factors: list = None) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Run a grid search across different numbers of factors and lookback periods.
    
    Parameters
    ----------
    excel_path : str
        Path to Excel file containing factor returns data
    n_factors_list : list of int
        List of different numbers of factors to try
    lookbacks_list : list of int
        List of different lookback periods to try
    allowed_factors : list, default=None
        List of factor names to consider. If None, all factors are used.
        
    Returns
    -------
    tuple
        (returns_grid, sharpe_grid, turnover_grid) - DataFrames containing results
    """
    # Initialize result grids
    returns_grid = pd.DataFrame(index=[f"{lb}m" for lb in lookbacks_list], 
                               columns=[f"{n}f" for n in n_factors_list])
    sharpe_grid = pd.DataFrame(index=[f"{lb}m" for lb in lookbacks_list], 
                              columns=[f"{n}f" for n in n_factors_list])
    turnover_grid = pd.DataFrame(index=[f"{lb}m" for lb in lookbacks_list], 
                                columns=[f"{n}f" for n in n_factors_list])
    
    total_combinations = len(n_factors_list) * len(lookbacks_list)
    completed = 0
    
    print(f"Running grid search across {total_combinations} combinations...")
    start_time = time.time()
    
    for lookback in lookbacks_list:
        for n_factors in n_factors_list:
            # Run strategy with current parameters
            returns, positions, _ = factor_timing_strategy(excel_path, n_top_factors=n_factors, 
                                                       lookback=lookback, allowed_factors=allowed_factors)
            
            # Calculate performance metrics
            ann_return, sharpe, turnover = calculate_performance_metrics(returns, positions)
            
            # Store results
            returns_grid.loc[f"{lookback}m", f"{n_factors}f"] = ann_return
            sharpe_grid.loc[f"{lookback}m", f"{n_factors}f"] = sharpe
            turnover_grid.loc[f"{lookback}m", f"{n_factors}f"] = turnover
            
            # Update progress
            completed += 1
            elapsed = time.time() - start_time
            avg_time = elapsed / completed
            remaining = avg_time * (total_combinations - completed)
            
            print(f"Completed {completed}/{total_combinations} combinations. " +
                  f"Estimated time remaining: {remaining:.1f}s", end="\r")
    
    print(f"\nGrid search completed in {time.time() - start_time:.1f} seconds.")
    
    return returns_grid, sharpe_grid, turnover_grid

def format_grid(grid: pd.DataFrame, is_return: bool = True) -> pd.DataFrame:
    """Format grid for display with appropriate formatting"""
    if is_return:
        return grid.applymap(lambda x: f"{x*100:.2f}%" if pd.notnull(x) else "N/A")
    else:
        return grid.applymap(lambda x: f"{x:.2f}" if pd.notnull(x) else "N/A")

if __name__ == "__main__":
    # Define grid search parameters
    n_factors_list = [1, 3, 5, 7, 10, 15]
    lookbacks_list = [24, 36, 60, 72, 90]
    
    # Load factor categories to filter only those with Max=1
    print("Loading factor categories from Step Factor Categories.xlsx...")
    factor_categories = pd.read_excel("Step Factor Categories.xlsx")
    allowed_factors = factor_categories[factor_categories["Max"] == 1]["Factor Name"].tolist()
    
    print(f"Found {len(allowed_factors)} factors with Max=1 that will be included in the grid search:")
    print(", ".join(allowed_factors))
    print()
    
    # Run grid search with filtered factors
    returns_grid, sharpe_grid, turnover_grid = run_grid_search("T2_Optimizer.xlsx", 
                                                          n_factors_list, 
                                                          lookbacks_list, 
                                                          allowed_factors=allowed_factors)
    
    # Format grids for display
    returns_formatted = format_grid(returns_grid, is_return=True)
    sharpe_formatted = format_grid(sharpe_grid, is_return=False)
    turnover_formatted = format_grid(turnover_grid, is_return=True)
    
    # Display results
    print("\n" + "="*80)
    print("ANNUALIZED RETURNS GRID")
    print("="*80)
    print("Rows: Lookback Periods, Columns: Number of Factors")
    print(tabulate(returns_formatted, headers='keys', tablefmt='grid'))
    
    print("\n" + "="*80)
    print("SHARPE RATIOS GRID")
    print("="*80)
    print("Rows: Lookback Periods, Columns: Number of Factors")
    print(tabulate(sharpe_formatted, headers='keys', tablefmt='grid'))
    
    print("\n" + "="*80)
    print("MONTHLY TURNOVER GRID")
    print("="*80)
    print("Rows: Lookback Periods, Columns: Number of Factors")
    print(tabulate(turnover_formatted, headers='keys', tablefmt='grid'))
    
    # Find and display best combinations
    max_return_idx = returns_grid.stack().idxmax()
    max_sharpe_idx = sharpe_grid.stack().idxmax()
    min_turnover_idx = turnover_grid.stack().idxmin()
    
    print("\n" + "="*80)
    print("BEST COMBINATIONS")
    print("="*80)
    print(f"Best Return: {returns_grid.stack().max()*100:.2f}% with {max_return_idx[1]} factors and {max_return_idx[0]} lookback")
    print(f"Best Sharpe: {sharpe_grid.stack().max():.2f} with {max_sharpe_idx[1]} factors and {max_sharpe_idx[0]} lookback")
    print(f"Lowest Turnover: {turnover_grid.stack().min()*100:.2f}% with {min_turnover_idx[1]} factors and {min_turnover_idx[0]} lookback")
